In [1]:
!pip install pandas openpyxl
import pandas as pd
import os

In [2]:
import pandas as pd
import os

In [2]:
import pandas as pd
import os



#Definir carpetas
base_dir = r"C:\Users\51943\Videos"
input_dir = os.path.join(base_dir, "Paises")
output_dir = os.path.join(base_dir, "Modificadas")




for archivo in os.listdir(input_dir):
    if archivo.endswith(".xlsx"):
        pais = os.path.splitext(archivo)[0]
        print(f"Procesando: {pais}")

        # Ruta completa del archivo
        ruta = os.path.join(input_dir, archivo)
        df = pd.read_excel(ruta)

        # aca le agrego una "a" antes de cada año para que luego con el reshape se pueda trabajar mejor
        df.columns = [f"a{col}" if str(col).isdigit() and len(str(col)) == 4 else col for col in df.columns]


        # Limpiar columnas que no necesito 
        
        df.rename(columns={"COUNTERPART_COUNTRY": "Country"}, inplace=True)

        # Crear variable Index que va representar 1 para  import y 2 para export 
        df["Index"] = df["INDICATOR"].map({
            "Imports of goods, Cost insurance freight (CIF), US dollar": 1,
            "Exports of goods, Free on board (FOB), US dollar": 2
        })


        # Reshape: aca se hace un doble proceso de reshape puesto que queremos que la variable Index este en dos columnas
        
        
        
        a_columns = [col for col in df.columns if col.startswith("a")]
        df_long = df.melt(id_vars=["Country", "Index"], value_vars=a_columns,
                          var_name="year", value_name="a")
        df_long["year"] = df_long["year"].str.extract(r"a(\d+)").astype(int)

        # Reshape: long → wide
        df_wide = df_long.pivot_table(index=["Country", "year"], columns="Index", values="a").reset_index()
        df_wide.columns.name = None
        df_wide.rename(columns={1: "Importaciones", 2: "Exportaciones"}, inplace=True)

        # Calcular VDI y Por: , VDI es la variable de comercio que es la suma y la de Por es el porcentaje
        
        
        
        df_wide["Importaciones"] = df_wide["Importaciones"].fillna(0)
        df_wide["Exportaciones"] = df_wide["Exportaciones"].fillna(0)
        df_wide["VDI"] = df_wide["Importaciones"] + df_wide["Exportaciones"]
        df_wide["T"] = df_wide.groupby("year")["VDI"].transform("sum")
        df_wide["Por"] = (df_wide["VDI"] / df_wide["T"]) * 100

        # Ahora se hace un orden  descendente donde se pondera primero a los que en porcentaje aportan más
        
        
    # Ordenar por año y Por descendente
        df_wide.sort_values(by=["year", "Por"], ascending=[True, False], inplace=True)

# Calcular acumulado por año
        df_wide["Por_cum"] = df_wide.groupby("year")["Por"].cumsum()

# Crear ranking dentro de cada año
        df_wide["Rank"] = df_wide.groupby("year")["Por"].rank(method="first", ascending=False)

# Filtrar: países dentro del 95% o dentro del top 50
        df_wide["Indicador"] = ((df_wide["Por_cum"] <= 95) | (df_wide["Rank"] <= 50)).astype(int)

# Aplicar el filtro final
        df_wide = df_wide[df_wide["Indicador"] == 1].copy()


        # se guarda
        pais_clean = pais.replace(",", "").replace(" ", "_")
        salida = os.path.join(output_dir, f"{pais_clean}_m.xlsx")
        df_wide.to_excel(salida, index=False)


Procesando: Albania
Procesando: Argentina
Procesando: Armenia
Procesando: Azerbaiyán
Procesando: Belarus, Republic of
Procesando: Belize
Procesando: Bosnia and Herzegovina
Procesando: Botswana
Procesando: Brazil
Procesando: China, People's Republic of
Procesando: Colombia
Procesando: Costa Rica
Procesando: Cuba
Procesando: Dominica
Procesando: Dominican Republic
Procesando: Ecuador
Procesando: El Salvador
Procesando: Equatorial Guinea, Republic of
Procesando: Fiji, Republic of
Procesando: Gabon
Procesando: Georgia
Procesando: Grenada
Procesando: Guatemala
Procesando: Indonesia
Procesando: Iraq
Procesando: Jamaica
Procesando: Kazakhstan, Republic of
Procesando: Libya
Procesando: Maldives
Procesando: Marshall Islands, Republic of the
Procesando: Mauritius
Procesando: Mexico
Procesando: Moldova, Republic of
Procesando: North Macedonia, Republic of
Procesando: Paraguay
Procesando: Peru
Procesando: South Africa
Procesando: St. Lucia
Procesando: St. Vincent and the Grenadines
Procesando: Sur

In [3]:
import pandas as pd
import os

# 📁 Ruta base
base_dir = r"C:\Users\51943\Videos"
input_dir = os.path.join(base_dir, "Modificadas")

# 🧾 Reemplazos especiales para contenido Y nombres de archivo
reemplazos_especiales = {
    "Netherlands, The": "Netherlands",
    "United States": "United States of America",
    "China, People's Republic of": "China",
    "Poland, Republic of": "Poland",
    "Egypt, Arab Republic of": "Egypt",
    "Türkiye, Republic of": "Turkey",
    "Iran, Islamic Republic of": "Iran",
    "Taiwan Province of China": "Taiwan",
    "Korea, Republic of": "Korea",
    "Kazakhstan, Republic of": "Kazakhstan",
    "Uzbekistan, Republic of": "Uzbekistan",
    "Azerbaijan, Republic of": "Azerbaijan",
    "Belgium": "Belgium and Luxembourg",
    "North Macedonia, Republic of": "North Macedonia",
    "Slovenia, Republic of": "Slovenia",
    "Croatia, Republic of": "Croatia",
    "Venezuela, República Bolivariana de": "Venezuela",
    "Czechoslovakia": "Czech Republic",
    "Hong Kong Special Administrative Region, People's Republic of China": "Hong Kong",
    "Netherlands Antilles": "Netherland Antilles",
    "Latvia, Republic of": "Latvia",
    "Madagascar, Republic of": "Madagascar",
    "Slovak Republic": "Slovakia",
    "Montenegro": "Serbia and Montenegro",
    "Serbia, Republic of": "Serbia and Montenegro",
    "Yemen, Republic of": "Yemen",
    "Vietnam": "Viet Nam",
    "Mozambique, Republic of": "Mozambique",
    "North Macedonia": "Macedonia (the former Yugoslav Rep. of)",
    "Kazakhstan": "Kazakstan",
    "Belarus, Republic of": "Belarus",
    "Libya": "Libyan Arab Jamahiriya",
    "Congo, Democratic Republic of the": "Congo (Democratic Republic of the)",
    "Bahamas,The": "Bahamas",
    "Bahrain, Kingdom of": "Bahrain",
    "Moldova Republic of": "Moldova, Rep.of",
    "St. Lucia": "Saint Lucia",
    "Equatorial Guinea Republic of":"Equatorial Guinea",
    "Marshall Islands Republic of the":"Marshall Islands",
    "St. Vincent and the Grenadines":"Saint Vincent and the Grenadines",
    "Korea, Democratic People's Republic of":"Korea, Dem. People's Rep. of"
}

# Sufijos genéricos solo para contenido (no nombres)
sufijos_genericos = [
    ", Republic of", ", Arab Republic of", ", Islamic Republic of",
    ", People's Republic of", ", República Bolivariana de"
]

def limpiar_contenido(pais):
    if pais in reemplazos_especiales:
        return reemplazos_especiales[pais]
    for sufijo in sufijos_genericos:
        if pais.endswith(sufijo):
            return pais.replace(sufijo, "")
    return pais

# 🔁 Procesar archivos
for archivo in os.listdir(input_dir):
    if archivo.endswith(".xlsx") and "distancia" not in archivo.lower():
        ruta = os.path.join(input_dir, archivo)
        try:
            df = pd.read_excel(ruta)

            # Limpiar columnas de texto (contenido)
            for col in df.columns:
                if df[col].dtype == "object":
                    df[col] = df[col].apply(lambda x: limpiar_contenido(x) if isinstance(x, str) else x)

            # Obtener nombre base sin extensión ni "_m"
            nombre_base = os.path.splitext(archivo)[0].replace("_m", "").replace("_", " ").strip()

            # Cambiar nombre solo si está en reemplazos especiales
            if nombre_base in reemplazos_especiales:
                nuevo_nombre = reemplazos_especiales[nombre_base].replace(" ", "_") + "_m.xlsx"
                nueva_ruta = os.path.join(input_dir, nuevo_nombre)

                # Guardar con nuevo nombre y eliminar original
                df.to_excel(nueva_ruta, index=False)
                os.remove(ruta)
                print(f"✅ Renombrado archivo: {archivo} → {nuevo_nombre}")
            else:
                # Guardar con el mismo nombre si no se renombra
                df.to_excel(ruta, index=False)
                print(f"✅ Reemplazos aplicados en: {archivo}")

        except Exception as e:
            print(f"❌ Error procesando {archivo}: {e}")


✅ Reemplazos aplicados en: Albania_m.xlsx
✅ Reemplazos aplicados en: Argentina_m.xlsx
✅ Reemplazos aplicados en: Armenia_m.xlsx
✅ Reemplazos aplicados en: Azerbaijan_m.xlsx
✅ Reemplazos aplicados en: Azerbaiyán_m.xlsx
✅ Reemplazos aplicados en: Belarus_m.xlsx
✅ Reemplazos aplicados en: Belarus_Republic_of_m.xlsx
✅ Reemplazos aplicados en: Belize_m.xlsx
✅ Reemplazos aplicados en: Bosnia_and_Herzegovina_m.xlsx
✅ Reemplazos aplicados en: Botswana_m.xlsx
✅ Reemplazos aplicados en: Brazil_m.xlsx
✅ Reemplazos aplicados en: China_m.xlsx
✅ Reemplazos aplicados en: China_People's_Republic_of_m.xlsx
✅ Reemplazos aplicados en: Colombia_m.xlsx
✅ Reemplazos aplicados en: Costa_Rica_m.xlsx
✅ Reemplazos aplicados en: Cuba_m.xlsx
✅ Reemplazos aplicados en: Dominican_Republic_m.xlsx
✅ Reemplazos aplicados en: Dominica_m.xlsx
✅ Reemplazos aplicados en: Ecuador_m.xlsx
✅ Reemplazos aplicados en: El_Salvador_m.xlsx
✅ Reemplazos aplicados en: Equatorial Guinea_m.xlsx
✅ Renombrado archivo: Equatorial_Guinea_

In [4]:

# === RUTA BASE ===
base_dir = r"C:\Users\51943\Videos"
ruta_diccionario = os.path.join(base_dir, "nombre.xlsx")
modificadas_dir = os.path.join(base_dir, "Modificadas")

# === 1. Crear diccionario de nombres a ISO ===
df_nombres = pd.read_excel(ruta_diccionario)

diccionario_paises = pd.concat([
    df_nombres[["paisi", "iso_o"]].rename(columns={"paisi": "pais", "iso_o": "iso"}),
    df_nombres[["paisj", "iso_d"]].rename(columns={"paisj": "pais", "iso_d": "iso"})
]).drop_duplicates().set_index("pais")["iso"].to_dict()

# === 2. Procesar archivos en carpeta Modificadas ===
for archivo in os.listdir(modificadas_dir):
    if archivo.endswith(".xlsx"):
        ruta = os.path.join(modificadas_dir, archivo)
        df = pd.read_excel(ruta)

        # Extraer nombre del país desde el nombre del archivo (ej. "China_m.xlsx" → "China")
        nombre_pais = os.path.splitext(archivo)[0].replace("_m", "").replace("_", " ").strip()

        # Asignar ISOi desde nombre del archivo
        df["ISOi"] = diccionario_paises.get(nombre_pais, nombre_pais)

        # Asignar ISOj desde columna Country
        df["ISOj"] = df["Country"].map(diccionario_paises).fillna(df["Country"])

        # Guardar archivo sobrescribiendo
        df.to_excel(ruta, index=False)
        print(f"✅ Archivo actualizado: {archivo}")


✅ Archivo actualizado: Albania_m.xlsx
✅ Archivo actualizado: Argentina_m.xlsx
✅ Archivo actualizado: Armenia_m.xlsx
✅ Archivo actualizado: Azerbaijan_m.xlsx
✅ Archivo actualizado: Azerbaiyán_m.xlsx
✅ Archivo actualizado: Belarus_m.xlsx
✅ Archivo actualizado: Belarus_Republic_of_m.xlsx
✅ Archivo actualizado: Belize_m.xlsx
✅ Archivo actualizado: Bosnia_and_Herzegovina_m.xlsx
✅ Archivo actualizado: Botswana_m.xlsx
✅ Archivo actualizado: Brazil_m.xlsx
✅ Archivo actualizado: China_m.xlsx
✅ Archivo actualizado: China_People's_Republic_of_m.xlsx
✅ Archivo actualizado: Colombia_m.xlsx
✅ Archivo actualizado: Costa_Rica_m.xlsx
✅ Archivo actualizado: Cuba_m.xlsx
✅ Archivo actualizado: Dominican_Republic_m.xlsx
✅ Archivo actualizado: Dominica_m.xlsx
✅ Archivo actualizado: Ecuador_m.xlsx
✅ Archivo actualizado: El_Salvador_m.xlsx
✅ Archivo actualizado: Equatorial Guinea_m.xlsx
✅ Archivo actualizado: Equatorial_Guinea_m.xlsx
✅ Archivo actualizado: Fiji_Republic_of_m.xlsx
✅ Archivo actualizado: Gabon_

In [5]:
import pandas as pd
import os

# 📁 Rutas
base_dir = r"C:\Users\51943\Videos"
input_dir = os.path.join(base_dir, "Modificadas")
output_dir = os.path.join(base_dir, "finales")
archivo_distancia = os.path.join(base_dir, "distancia.xlsx")

# 📦 Cargar base de distancia
df_dist = pd.read_excel(archivo_distancia)

# 🔁 Procesar cada archivo en Modificadas
for archivo in os.listdir(input_dir):
    if archivo.endswith(".xlsx") and "distancia" not in archivo.lower():
        ruta_modificada = os.path.join(input_dir, archivo)
        try:
            # Cargar archivo
            df_mod = pd.read_excel(ruta_modificada)

            # Asegurar que columnas ISO existen
            if "ISOi" not in df_mod.columns or "ISOj" not in df_mod.columns:
                print(f"⚠️ Archivo sin columnas ISOi/ISOj: {archivo}")
                continue

            # Extraer ISOi único del archivo
            iso_i_unico = df_mod["ISOi"].dropna().unique()
            if len(iso_i_unico) != 1:
                print(f"❌ ISOi múltiple o faltante en: {archivo}")
                continue
            iso_i = iso_i_unico[0]

            # Filtrar base de distancia solo para ese ISOi
            df_dist_filtrado = df_dist[df_dist["iso_o"] == iso_i]

            # Hacer merge por ISOj ↔ iso_d
            df_merge = df_mod.merge(
                df_dist_filtrado[["iso_d", "distcapij", "frontera", "areai", "areaj", "mari", "marj"]],
                left_on="ISOj",
                right_on="iso_d",
                how="left"
            )

            # Eliminar iso_d redundante
            df_merge.drop(columns=["iso_d"], inplace=True)

            # Guardar en carpeta finales
            salida = os.path.join(output_dir, archivo)
            df_merge.to_excel(salida, index=False)
            print(f"✅ Merge realizado y guardado: {archivo}")

        except Exception as e:
            print(f"❌ Error procesando {archivo}: {e}")




✅ Merge realizado y guardado: Albania_m.xlsx
✅ Merge realizado y guardado: Argentina_m.xlsx
✅ Merge realizado y guardado: Armenia_m.xlsx
✅ Merge realizado y guardado: Azerbaijan_m.xlsx
✅ Merge realizado y guardado: Azerbaiyán_m.xlsx
✅ Merge realizado y guardado: Belarus_m.xlsx
✅ Merge realizado y guardado: Belarus_Republic_of_m.xlsx
✅ Merge realizado y guardado: Belize_m.xlsx
✅ Merge realizado y guardado: Bosnia_and_Herzegovina_m.xlsx
✅ Merge realizado y guardado: Botswana_m.xlsx
✅ Merge realizado y guardado: Brazil_m.xlsx
✅ Merge realizado y guardado: China_m.xlsx
✅ Merge realizado y guardado: China_People's_Republic_of_m.xlsx
✅ Merge realizado y guardado: Colombia_m.xlsx
✅ Merge realizado y guardado: Costa_Rica_m.xlsx
✅ Merge realizado y guardado: Cuba_m.xlsx
✅ Merge realizado y guardado: Dominican_Republic_m.xlsx
✅ Merge realizado y guardado: Dominica_m.xlsx
✅ Merge realizado y guardado: Ecuador_m.xlsx
✅ Merge realizado y guardado: El_Salvador_m.xlsx
✅ Merge realizado y guardado: Eq

In [4]:
import pandas as pd
import os

base_dir = r"C:\Users\51943\Videos"
archivo_gdp = os.path.join(base_dir, "gdp.xlsx")

# Cargar datos
df = pd.read_excel(archivo_gdp)

# Convertir de wide a long: columnas año → variable "year"
df_long = df.melt(
    id_vars=["Country Name", "Country Code"],
    var_name="year",
    value_name="gdp"
)

# Convertir año a número
df_long["year"] = df_long["year"].astype(int)
df_long.rename(columns={"Country Code": "country_code"}, inplace=True)

# Guardar si deseas
df_long.to_excel(os.path.join(base_dir, "gdp_long.xlsx"), index=False)
print("✅ GDP convertido de wide a long.")



✅ GDP convertido de wide a long.


In [6]:

# 📁 Ruta carpeta
base_dir = r"C:\Users\51943\Videos"
finales_dir = os.path.join(base_dir, "finales")

# 📄 Lista de DataFrames
dfs = []

# 🔁 Leer todos los archivos .xlsx
for archivo in os.listdir(finales_dir):
    if archivo.endswith(".xlsx"):
        ruta = os.path.join(finales_dir, archivo)
        try:
            df = pd.read_excel(ruta)
            df["archivo_origen"] = archivo  # opcional: saber de dónde vino cada fila
            dfs.append(df)
        except Exception as e:
            print(f"❌ Error leyendo {archivo}: {e}")

# 🧩 Concatenar
df_final = pd.concat(dfs, ignore_index=True)

# 💾 Guardar
output_path = os.path.join(base_dir, "completo_final.xlsx")
df_final.to_excel(output_path, index=False)
print(f"✅ Base consolidada guardada como: {output_path}")


✅ Base consolidada guardada como: C:\Users\51943\Videos\completo_final.xlsx


In [9]:
import pandas as pd
import os

# 📁 Rutas
base_dir = r"C:\Users\51943\Videos"
archivo_completo = os.path.join(base_dir, "finales", "completo_final.xlsx")
archivo_gdp = os.path.join(base_dir, "gdp_long.xlsx")
output_dir = os.path.join(base_dir, "final")
os.makedirs(output_dir, exist_ok=True)

# 📦 Cargar bases
df_completo = pd.read_excel(archivo_completo)
df_gdp = pd.read_excel(archivo_gdp)

# 🧼 Renombrar y limpiar nombres
df_gdp.rename(columns={"Country Name": "country"}, inplace=True)
reemplazos_nombres = {
    "Turkiye": "Turkey",
    "Iran, Islamic Rep.": "Iran",
    "Venezuela, RB": "Venezuela",
    "Vietnam": "Viet Nam",
    "North Macedonia": "Macedonia (the former Yugoslav Rep. of)",
    "Slovak Republic": "Slovakia",
    "Czechia": "Czech Republic",
    "Yemen, Rep.": "Yemen",
    "Syrian Arab Republic": "Syrian Arab Republic",
    "St. Lucia": "Saint Lucia",
    "St. Kitts and Nevis": "Saint Kitts and Nevis",
    "St. Vincent and the Grenadines": "Saint Vincent and the Grenadines",
    "Lao PDR": "Lao People's Democratic Republic",
    "Kyrgyz Republic": "Kyrgyzstan",
    "Micronesia, Fed. Sts.": "Micronesia (Federated States of)",
    "Congo, Dem. Rep.": "Congo (Democratic Republic of the)",
    "Congo, Rep.": "Congo",
    "Gambia, The": "Gambia",
    "West Bank and Gaza": "Palestine, State of",
    "Macao SAR, China": "Macao",
    "Hong Kong SAR, China": "Hong Kong",
    "Bonaire, Sint Eustatius and Saba": "Bonaire",
    "Curacao": "Curaçao",
    "Sint Maarten (Dutch part)": "Sint Maarten",
    "St. Martin (French part)": "Saint Martin (French part)",
    "Eswatini": "Swaziland",
    "Kosovo": "Kosovo",
    "Faroe Islands": "Faroe Islands",
    "American Samoa": "American Samoa",
    "Greenland": "Greenland",
    "Puerto Rico": "Puerto Rico",
    "Guam": "Guam",
    "Cabo Verde": "Cape Verde",
    "United States": "United States of America",
    "Korea, Rep.": "Korea",
    "Korea, Dem. People's Rep.": "Korea, Dem. People's Rep. of",
    "Kazakhstan": "Kazakstan",
    "Belgium": "Belgium and Luxembourg"
}
df_gdp["country"] = df_gdp["country"].replace(reemplazos_nombres)

# 🧬 1. Merge para obtener PBI_j (por Country)
df_merged = df_completo.merge(
    df_gdp[["country", "year", "gdp"]],
    how="left",
    left_on=["Country", "year"],
    right_on=["country", "year"]
).rename(columns={"gdp": "PBI_j"})

# 🧬 2. Merge adicional para obtener PBI_i (por ISOi)
df_merged = df_merged.merge(
    df_gdp[["country_code", "year", "gdp"]],
    how="left",
    left_on=["ISOi", "year"],
    right_on=["country_code", "year"]
).rename(columns={"gdp": "PBI_i"})

# 🧽 Limpiar columnas auxiliares
df_merged.drop(columns=["country", "country_code"], inplace=True)

# 💾 Guardar archivo final
archivo_salida = os.path.join(output_dir, "completo_final_con_pbi_i_y_j.xlsx")
df_merged.to_excel(archivo_salida, index=False)
print(f"✅ Archivo con PBI_i y PBI_j guardado en: {archivo_salida}")




✅ Archivo con PBI_i y PBI_j guardado en: C:\Users\51943\Videos\final\completo_final_con_pbi_i_y_j.xlsx


In [7]:

# 📁 Ruta base y archivo
base_dir = r"C:\Users\51943\Videos"
archivo_pob = os.path.join(base_dir, "pob.xlsx")

# 📦 Cargar datos
df = pd.read_excel(archivo_pob)

# 🔄 Convertir de wide a long
df_long = df.melt(
    id_vars=["Country Name", "Country Code"],
    var_name="year",
    value_name="population"
)

# 🧼 Convertir año a entero
df_long["year"] = df_long["year"].astype(int)
df_long.rename(columns={"Country Code": "country_code"}, inplace=True)

# 💾 Guardar resultado
df_long.to_excel(os.path.join(base_dir, "pob_long.xlsx"), index=False)
print("✅ Población convertida de wide a long.")


✅ Población convertida de wide a long.


In [10]:


# 📁 Rutas
base_dir = r"C:\Users\51943\Videos"
archivo_completo = os.path.join(base_dir, "final", "completo_final_con_pbi_i_y_j.xlsx")
archivo_pob = os.path.join(base_dir, "pob_long.xlsx")
output_dir = os.path.join(base_dir, "final")
os.makedirs(output_dir, exist_ok=True)

# 📦 Cargar bases
df_completo = pd.read_excel(archivo_completo)
df_pob = pd.read_excel(archivo_pob)

# 🧼 Renombrar
df_pob.rename(columns={"Country Name": "country"}, inplace=True)

# 🧽 Reemplazos de nombres alternativos
reemplazos_nombres = {
    "Turkiye": "Turkey",
    "Iran, Islamic Rep.": "Iran",
    "Venezuela, RB": "Venezuela",
    "Vietnam": "Viet Nam",
    "North Macedonia": "Macedonia (the former Yugoslav Rep. of)",
    "Slovak Republic": "Slovakia",
    "Czechia": "Czech Republic",
    "Yemen, Rep.": "Yemen",
    "Syrian Arab Republic": "Syrian Arab Republic",
    "St. Lucia": "Saint Lucia",
    "St. Kitts and Nevis": "Saint Kitts and Nevis",
    "St. Vincent and the Grenadines": "Saint Vincent and the Grenadines",
    "Lao PDR": "Lao People's Democratic Republic",
    "Kyrgyz Republic": "Kyrgyzstan",
    "Micronesia, Fed. Sts.": "Micronesia (Federated States of)",
    "Congo, Dem. Rep.": "Congo (Democratic Republic of the)",
    "Congo, Rep.": "Congo",
    "Gambia, The": "Gambia",
    "West Bank and Gaza": "Palestine, State of",
    "Macao SAR, China": "Macao",
    "Hong Kong SAR, China": "Hong Kong",
    "Bonaire, Sint Eustatius and Saba": "Bonaire",
    "Curacao": "Curaçao",
    "Sint Maarten (Dutch part)": "Sint Maarten",
    "St. Martin (French part)": "Saint Martin (French part)",
    "Eswatini": "Swaziland",
    "Kosovo": "Kosovo",
    "Faroe Islands": "Faroe Islands",
    "American Samoa": "American Samoa",
    "Greenland": "Greenland",
    "Puerto Rico": "Puerto Rico",
    "Guam": "Guam",
    "Cabo Verde": "Cape Verde",
    "United States": "United States of America",
    "Korea, Rep.": "Korea",
    "Korea, Dem. People's Rep.": "Korea, Dem. People's Rep. of",
    "Kazakhstan": "Kazakstan",
    "Belgium": "Belgium and Luxembourg",
    "Egypt, Arab Rep.": "Egypt"
}
df_pob["country"] = df_pob["country"].replace(reemplazos_nombres)

# 🧬 1. Merge para obtener Población_j
df_merge = df_completo.merge(
    df_pob[["country", "year", "population"]],
    how="left",
    left_on=["Country", "year"],
    right_on=["country", "year"]
).rename(columns={"population": "Población_j"})

# 🧬 2. Merge adicional para obtener Población_i
df_merge = df_merge.merge(
    df_pob[["country_code", "year", "population"]],
    how="left",
    left_on=["ISOi", "year"],
    right_on=["country_code", "year"]
).rename(columns={"population": "Población_i"})

# 🧽 Limpiar columnas auxiliares


archivo_salida = os.path.join(output_dir, "completo_pob_gdp.xlsx")
df_merge.to_excel(archivo_salida, index=False)
print(f"✅ Archivo con Población_j y Población_i guardado en: {archivo_salida}")



✅ Archivo con Población_j y Población_i guardado en: C:\Users\51943\Videos\final\completo_pob_gdp.xlsx


In [11]:



archivo_completo = os.path.join(base_dir, "final", "completo_pob_gdp.xlsx")

# 📦 Cargar archivo
df = pd.read_excel(archivo_completo)

# 🧼 Limpiar columna gdp
df["PBI_i"] = df["PBI_i"].replace(["..", "..."], 0)       # Reemplazar ".." y "..."
df["PBI_i"] = pd.to_numeric(df["PBI_i"], errors="coerce") # Convertir a numérico
df["PBI_i"] = df["PBI_i"].fillna(0)                       # Reemplazar NaN por 0

# ➗ Calcular Tao/pbi
df["Tao/pbi"] = df["VDI"] / df["PBI_i"]

# 💾 Guardar archivo final
output_path = os.path.join(base_dir, "final", "base_final.xlsx")
df.to_excel(output_path, index=False)

print(f"✅ GDP limpiado y 'Tao/pbi' calculado. Archivo guardado en: {output_path}")



✅ GDP limpiado y 'Tao/pbi' calculado. Archivo guardado en: C:\Users\51943\Videos\final\base_final.xlsx


In [4]:

# 📁 Ruta base y archivo
base_dir = r"C:\Users\51943\Videos"
archivo_pob = os.path.join(base_dir, "grow.xlsx")

# 📦 Cargar datos
df = pd.read_excel(archivo_pob)

# 🔄 Convertir de wide a long
df_long = df.melt(
    id_vars=["Country Name", "Country Code"],
    var_name="year",
    value_name="grow"
)

# 🧼 Convertir año a entero
df_long["year"] = df_long["year"].astype(int)
df_long.rename(columns={"Country Code": "country_code"}, inplace=True)

# 💾 Guardar resultado
df_long.to_excel(os.path.join(base_dir, "grow_long.xlsx"), index=False)
print("✅ Población convertida de wide a long.")

✅ Población convertida de wide a long.


In [12]:

# 📁 Rutas
base_dir = r"C:\Users\51943\Videos"
archivo_completo = os.path.join(base_dir, "final", "base_final.xlsx")
archivo_pob = os.path.join(base_dir, "grow_long.xlsx")
output_dir = os.path.join(base_dir, "final")
os.makedirs(output_dir, exist_ok=True)

# 📦 Cargar bases
df_completo = pd.read_excel(archivo_completo)
df_pob = pd.read_excel(archivo_pob)

# 🧼 Renombrar
df_pob.rename(columns={"Country Name": "country"}, inplace=True)

# 🧽 Reemplazos de nombres alternativos
reemplazos_nombres = {
    "Turkiye": "Turkey",
    "Iran, Islamic Rep.": "Iran",
    "Venezuela, RB": "Venezuela",
    "Vietnam": "Viet Nam",
    "North Macedonia": "Macedonia (the former Yugoslav Rep. of)",
    "Slovak Republic": "Slovakia",
    "Czechia": "Czech Republic",
    "Yemen, Rep.": "Yemen",
    "Syrian Arab Republic": "Syrian Arab Republic",
    "St. Lucia": "Saint Lucia",
    "St. Kitts and Nevis": "Saint Kitts and Nevis",
    "St. Vincent and the Grenadines": "Saint Vincent and the Grenadines",
    "Lao PDR": "Lao People's Democratic Republic",
    "Kyrgyz Republic": "Kyrgyzstan",
    "Micronesia, Fed. Sts.": "Micronesia (Federated States of)",
    "Congo, Dem. Rep.": "Congo (Democratic Republic of the)",
    "Congo, Rep.": "Congo",
    "Gambia, The": "Gambia",
    "West Bank and Gaza": "Palestine, State of",
    "Macao SAR, China": "Macao",
    "Hong Kong SAR, China": "Hong Kong",
    "Bonaire, Sint Eustatius and Saba": "Bonaire",
    "Curacao": "Curaçao",
    "Sint Maarten (Dutch part)": "Sint Maarten",
    "St. Martin (French part)": "Saint Martin (French part)",
    "Eswatini": "Swaziland",
    "Kosovo": "Kosovo",
    "Faroe Islands": "Faroe Islands",
    "American Samoa": "American Samoa",
    "Greenland": "Greenland",
    "Puerto Rico": "Puerto Rico",
    "Guam": "Guam",
    "Cabo Verde": "Cape Verde",
    "United States": "United States of America",
    "Korea, Rep.": "Korea",
    "Korea, Dem. People's Rep.": "Korea, Dem. People's Rep. of",
    "Kazakhstan": "Kazakstan",
    "Belgium": "Belgium and Luxembourg",
    "Egypt, Arab Rep.": "Egypt"
}
df_pob["country"] = df_pob["country"].replace(reemplazos_nombres)

# 🧬 1. Merge para obtener Población_j
df_merge = df_completo.merge(
    df_pob[["country", "year", "grow"]],
    how="left",
    left_on=["Country", "year"],
    right_on=["country", "year"]
).rename(columns={"grow": "grow_j"})

# 🧬 2. Merge adicional para obtener Población_i
df_merge = df_merge.merge(
    df_pob[["country_code", "year", "grow"]],
    how="left",
    left_on=["ISOi", "year"],
    right_on=["country_code", "year"]
).rename(columns={"grow": "grow_i"})

# 🧽 Limpiar columnas auxiliares


archivo_salida = os.path.join(output_dir, "completo_grow.xlsx")
df_merge.to_excel(archivo_salida, index=False)
print(f"✅ Archivo con grow_j y grow_i guardado en: {archivo_salida}")



✅ Archivo con grow_j y grow_i guardado en: C:\Users\51943\Videos\final\completo_grow.xlsx


In [6]:

# 📁 Ruta base y archivo
base_dir = r"C:\Users\51943\Videos"
archivo_pob = os.path.join(base_dir, "inflow.xlsx")

# 📦 Cargar datos
df = pd.read_excel(archivo_pob)

# 🔄 Convertir de wide a long
df_long = df.melt(
    id_vars=["Country Name", "Country Code"],
    var_name="year",
    value_name="inflow"
)

# 🧼 Convertir año a entero
df_long["year"] = df_long["year"].astype(int)
df_long.rename(columns={"Country Code": "country_code"}, inplace=True)

# 💾 Guardar resultado
df_long.to_excel(os.path.join(base_dir, "inflow_long.xlsx"), index=False)
print("✅ Población convertida de wide a long.")

✅ Población convertida de wide a long.


In [13]:

# 📁 Rutas
base_dir = r"C:\Users\51943\Videos"
archivo_completo = os.path.join(base_dir, "final", "completo_grow.xlsx")
archivo_pob = os.path.join(base_dir, "inflow_long.xlsx")
output_dir = os.path.join(base_dir, "final")
os.makedirs(output_dir, exist_ok=True)

# 📦 Cargar bases
df_completo = pd.read_excel(archivo_completo)
df_pob = pd.read_excel(archivo_pob)

# 🧼 Renombrar
df_pob.rename(columns={"Country Name": "country"}, inplace=True)

# 🧽 Reemplazos de nombres alternativos
reemplazos_nombres = {
    "Turkiye": "Turkey",
    "Iran, Islamic Rep.": "Iran",
    "Venezuela, RB": "Venezuela",
    "Vietnam": "Viet Nam",
    "North Macedonia": "Macedonia (the former Yugoslav Rep. of)",
    "Slovak Republic": "Slovakia",
    "Czechia": "Czech Republic",
    "Yemen, Rep.": "Yemen",
    "Syrian Arab Republic": "Syrian Arab Republic",
    "St. Lucia": "Saint Lucia",
    "St. Kitts and Nevis": "Saint Kitts and Nevis",
    "St. Vincent and the Grenadines": "Saint Vincent and the Grenadines",
    "Lao PDR": "Lao People's Democratic Republic",
    "Kyrgyz Republic": "Kyrgyzstan",
    "Micronesia, Fed. Sts.": "Micronesia (Federated States of)",
    "Congo, Dem. Rep.": "Congo (Democratic Republic of the)",
    "Congo, Rep.": "Congo",
    "Gambia, The": "Gambia",
    "West Bank and Gaza": "Palestine, State of",
    "Macao SAR, China": "Macao",
    "Hong Kong SAR, China": "Hong Kong",
    "Bonaire, Sint Eustatius and Saba": "Bonaire",
    "Curacao": "Curaçao",
    "Sint Maarten (Dutch part)": "Sint Maarten",
    "St. Martin (French part)": "Saint Martin (French part)",
    "Eswatini": "Swaziland",
    "Kosovo": "Kosovo",
    "Faroe Islands": "Faroe Islands",
    "American Samoa": "American Samoa",
    "Greenland": "Greenland",
    "Puerto Rico": "Puerto Rico",
    "Guam": "Guam",
    "Cabo Verde": "Cape Verde",
    "United States": "United States of America",
    "Korea, Rep.": "Korea",
    "Korea, Dem. People's Rep.": "Korea, Dem. People's Rep. of",
    "Kazakhstan": "Kazakstan",
    "Belgium": "Belgium and Luxembourg",
    "Egypt, Arab Rep.": "Egypt"
}
df_pob["country"] = df_pob["country"].replace(reemplazos_nombres)

# 🧬 1. Merge para obtener Población_j
df_merge = df_completo.merge(
    df_pob[["country", "year", "inflow"]],
    how="left",
    left_on=["Country", "year"],
    right_on=["country", "year"]
).rename(columns={"inflow": "inflow_j"})

# 🧬 2. Merge adicional para obtener Población_i
df_merge = df_merge.merge(
    df_pob[["country_code", "year", "inflow"]],
    how="left",
    left_on=["ISOi", "year"],
    right_on=["country_code", "year"]
).rename(columns={"inflow": "inflow_i"})

# 🧽 Limpiar columnas auxiliares


archivo_salida = os.path.join(output_dir, "completo_inflow.xlsx")
df_merge.to_excel(archivo_salida, index=False)
print(f"✅ Archivo con inflow_j y inflow_i guardado en: {archivo_salida}")


✅ Archivo con inflow_j y inflow_i guardado en: C:\Users\51943\Videos\final\completo_inflow.xlsx


In [8]:

# 📁 Ruta base y archivo
base_dir = r"C:\Users\51943\Videos"
archivo_pob = os.path.join(base_dir, "ind.xlsx")

# 📦 Cargar datos
df = pd.read_excel(archivo_pob)

# 🔄 Convertir de wide a long
df_long = df.melt(
    id_vars=["Country Name", "Country Code"],
    var_name="year",
    value_name="ind"
)

# 🧼 Convertir año a entero
df_long["year"] = df_long["year"].astype(int)
df_long.rename(columns={"Country Code": "country_code"}, inplace=True)

# 💾 Guardar resultado
df_long.to_excel(os.path.join(base_dir, "ind_long.xlsx"), index=False)
print("✅ Población convertida de wide a long.")

✅ Población convertida de wide a long.


In [14]:

# 📁 Rutas
base_dir = r"C:\Users\51943\Videos"
archivo_completo = os.path.join(base_dir, "final", "completo_inflow.xlsx")
archivo_pob = os.path.join(base_dir, "ind_long.xlsx")
output_dir = os.path.join(base_dir, "final")
os.makedirs(output_dir, exist_ok=True)

# 📦 Cargar bases
df_completo = pd.read_excel(archivo_completo)
df_pob = pd.read_excel(archivo_pob)

# 🧼 Renombrar
df_pob.rename(columns={"Country Name": "country"}, inplace=True)

# 🧽 Reemplazos de nombres alternativos
reemplazos_nombres = {
    "Turkiye": "Turkey",
    "Iran, Islamic Rep.": "Iran",
    "Venezuela, RB": "Venezuela",
    "Vietnam": "Viet Nam",
    "North Macedonia": "Macedonia (the former Yugoslav Rep. of)",
    "Slovak Republic": "Slovakia",
    "Czechia": "Czech Republic",
    "Yemen, Rep.": "Yemen",
    "Syrian Arab Republic": "Syrian Arab Republic",
    "St. Lucia": "Saint Lucia",
    "St. Kitts and Nevis": "Saint Kitts and Nevis",
    "St. Vincent and the Grenadines": "Saint Vincent and the Grenadines",
    "Lao PDR": "Lao People's Democratic Republic",
    "Kyrgyz Republic": "Kyrgyzstan",
    "Micronesia, Fed. Sts.": "Micronesia (Federated States of)",
    "Congo, Dem. Rep.": "Congo (Democratic Republic of the)",
    "Congo, Rep.": "Congo",
    "Gambia, The": "Gambia",
    "West Bank and Gaza": "Palestine, State of",
    "Macao SAR, China": "Macao",
    "Hong Kong SAR, China": "Hong Kong",
    "Bonaire, Sint Eustatius and Saba": "Bonaire",
    "Curacao": "Curaçao",
    "Sint Maarten (Dutch part)": "Sint Maarten",
    "St. Martin (French part)": "Saint Martin (French part)",
    "Eswatini": "Swaziland",
    "Kosovo": "Kosovo",
    "Faroe Islands": "Faroe Islands",
    "American Samoa": "American Samoa",
    "Greenland": "Greenland",
    "Puerto Rico": "Puerto Rico",
    "Guam": "Guam",
    "Cabo Verde": "Cape Verde",
    "United States": "United States of America",
    "Korea, Rep.": "Korea",
    "Korea, Dem. People's Rep.": "Korea, Dem. People's Rep. of",
    "Kazakhstan": "Kazakstan",
    "Belgium": "Belgium and Luxembourg",
    "Egypt, Arab Rep.": "Egypt"
}
df_pob["country"] = df_pob["country"].replace(reemplazos_nombres)

# 🧬 1. Merge para obtener Población_j
df_merge = df_completo.merge(
    df_pob[["country", "year", "ind"]],
    how="left",
    left_on=["Country", "year"],
    right_on=["country", "year"]
).rename(columns={"ind": "ind_j"})

# 🧬 2. Merge adicional para obtener Población_i
df_merge = df_merge.merge(
    df_pob[["country_code", "year", "ind"]],
    how="left",
    left_on=["ISOi", "year"],
    right_on=["country_code", "year"]
).rename(columns={"ind": "ind_i"})

# 🧽 Limpiar columnas auxiliares


archivo_salida = os.path.join(output_dir, "completo_ind.xlsx")
df_merge.to_excel(archivo_salida, index=False)
print(f"✅ Archivo con ind_j y ind_i guardado en: {archivo_salida}")


C:\Users\51943\AppData\Local\Temp\ipykernel_24632\3281867756.py:60: FutureWarning: Passing 'suffixes' which cause duplicate columns {'country_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_merge = df_completo.merge(
C:\Users\51943\AppData\Local\Temp\ipykernel_24632\3281867756.py:68: FutureWarning: Passing 'suffixes' which cause duplicate columns {'country_code_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_merge = df_merge.merge(


✅ Archivo con ind_j y ind_i guardado en: C:\Users\51943\Videos\final\completo_ind.xlsx


In [10]:

# 📁 Ruta base y archivo
base_dir = r"C:\Users\51943\Videos"
archivo_pob = os.path.join(base_dir, "inflation.xlsx")

# 📦 Cargar datos
df = pd.read_excel(archivo_pob)

# 🔄 Convertir de wide a long
df_long = df.melt(
    id_vars=["Country Name", "Country Code"],
    var_name="year",
    value_name="inflation"
)

# 🧼 Convertir año a entero
df_long["year"] = df_long["year"].astype(int)
df_long.rename(columns={"Country Code": "country_code"}, inplace=True)

# 💾 Guardar resultado
df_long.to_excel(os.path.join(base_dir, "inflation_long.xlsx"), index=False)
print("✅ Población convertida de wide a long.")

✅ Población convertida de wide a long.


In [15]:

# 📁 Rutas
base_dir = r"C:\Users\51943\Videos"
archivo_completo = os.path.join(base_dir, "final", "completo_ind.xlsx")
archivo_pob = os.path.join(base_dir, "inflation_long.xlsx")
output_dir = os.path.join(base_dir, "final")
os.makedirs(output_dir, exist_ok=True)

# 📦 Cargar bases
df_completo = pd.read_excel(archivo_completo)
df_pob = pd.read_excel(archivo_pob)

# 🧼 Renombrar
df_pob.rename(columns={"Country Name": "country"}, inplace=True)

# 🧽 Reemplazos de nombres alternativos
reemplazos_nombres = {
    "Turkiye": "Turkey",
    "Iran, Islamic Rep.": "Iran",
    "Venezuela, RB": "Venezuela",
    "Vietnam": "Viet Nam",
    "North Macedonia": "Macedonia (the former Yugoslav Rep. of)",
    "Slovak Republic": "Slovakia",
    "Czechia": "Czech Republic",
    "Yemen, Rep.": "Yemen",
    "Syrian Arab Republic": "Syrian Arab Republic",
    "St. Lucia": "Saint Lucia",
    "St. Kitts and Nevis": "Saint Kitts and Nevis",
    "St. Vincent and the Grenadines": "Saint Vincent and the Grenadines",
    "Lao PDR": "Lao People's Democratic Republic",
    "Kyrgyz Republic": "Kyrgyzstan",
    "Micronesia, Fed. Sts.": "Micronesia (Federated States of)",
    "Congo, Dem. Rep.": "Congo (Democratic Republic of the)",
    "Congo, Rep.": "Congo",
    "Gambia, The": "Gambia",
    "West Bank and Gaza": "Palestine, State of",
    "Macao SAR, China": "Macao",
    "Hong Kong SAR, China": "Hong Kong",
    "Bonaire, Sint Eustatius and Saba": "Bonaire",
    "Curacao": "Curaçao",
    "Sint Maarten (Dutch part)": "Sint Maarten",
    "St. Martin (French part)": "Saint Martin (French part)",
    "Eswatini": "Swaziland",
    "Kosovo": "Kosovo",
    "Faroe Islands": "Faroe Islands",
    "American Samoa": "American Samoa",
    "Greenland": "Greenland",
    "Puerto Rico": "Puerto Rico",
    "Guam": "Guam",
    "Cabo Verde": "Cape Verde",
    "United States": "United States of America",
    "Korea, Rep.": "Korea",
    "Korea, Dem. People's Rep.": "Korea, Dem. People's Rep. of",
    "Kazakhstan": "Kazakstan",
    "Belgium": "Belgium and Luxembourg",
    "Egypt, Arab Rep.": "Egypt"
}
df_pob["country"] = df_pob["country"].replace(reemplazos_nombres)

# 🧬 1. Merge para obtener Población_j
df_merge = df_completo.merge(
    df_pob[["country", "year", "inflation"]],
    how="left",
    left_on=["Country", "year"],
    right_on=["country", "year"]
).rename(columns={"inflation": "inflation_j"})

# 🧬 2. Merge adicional para obtener Población_i
df_merge = df_merge.merge(
    df_pob[["country_code", "year", "inflation"]],
    how="left",
    left_on=["ISOi", "year"],
    right_on=["country_code", "year"]
).rename(columns={"inflation": "inflation_i"})

# 🧽 Limpiar columnas auxiliares


archivo_salida = os.path.join(output_dir, "completo_inflation.xlsx")
df_merge.to_excel(archivo_salida, index=False)
print(f"✅ Archivo con inflation_j y inflation_i guardado en: {archivo_salida}")

✅ Archivo con inflation_j y inflation_i guardado en: C:\Users\51943\Videos\final\completo_inflation.xlsx


In [2]:

# 📁 Ruta base y archivo
base_dir = r"C:\Users\51943\Videos"
archivo_pob = os.path.join(base_dir, "pbi_percap.xlsx")

# 📦 Cargar datos
df = pd.read_excel(archivo_pob)

# 🔄 Convertir de wide a long
df_long = df.melt(
    id_vars=["Country Name", "Country Code"],
    var_name="year",
    value_name="pbi_percap"
)

# 🧼 Convertir año a entero
df_long["year"] = df_long["year"].astype(int)
df_long.rename(columns={"Country Code": "country_code"}, inplace=True)

# 💾 Guardar resultado
df_long.to_excel(os.path.join(base_dir, "pbi_percap_long.xlsx"), index=False)
print("✅ Población convertida de wide a long.")

✅ Población convertida de wide a long.


In [3]:
base_dir = r"C:\Users\51943\Videos"
archivo_pob = os.path.join(base_dir, "pbi_percap_long.xlsx")

df = pd.read_excel(archivo_pob)
df



,Country Name,country_code,year,pbi_percap
0,Afghanistan,AFG,1981,11937581
1,Afghanistan,AFG,1981,652860
2,Afghanistan,AFG,1981,180000
3,Afghanistan,AFG,1981,..
4,Afghanistan,AFG,1981,..
...,...,...,...,...
104055,Zimbabwe,ZWE,2024,..
104056,Zimbabwe,ZWE,2024,..
104057,Zimbabwe,ZWE,2024,..
104058,Zimbabwe,ZWE,2024,..


In [4]:
base_dir = r"C:\Users\51943\Videos"
archivo_completo = os.path.join(base_dir, "final", "completo_inflation.xlsx")
df_completo = pd.read_excel(archivo_completo)

df_completo 


,Country,year,Importaciones,Exportaciones,VDI,T,Por,Por_cum,Indicador,ISOi,...,PBI_i,country_x,Población_i,Tao/pbi,country_code_y,grow_i,inflow_i,ind_i,country_code,inflation_i
0,Italy,1981,25.850000,40.818182,66.668182,389.489336,17.116818,17.116818,1,ALB,...,1.808177e+09,Italy,2726056.0,3.687038e-08,ALB,5.745635,0,..,ALB,-2.139572
1,Romania,1981,20.064000,25.190909,45.254909,389.489336,11.619037,28.735855,1,ALB,...,1.808177e+09,Romania,2726056.0,2.502792e-08,ALB,5.745635,0,..,ALB,-2.139572
2,Poland,1981,19.217000,24.354545,43.571545,389.489336,11.186839,39.922694,1,ALB,...,1.808177e+09,Poland,2726056.0,2.409694e-08,ALB,5.745635,0,..,ALB,-2.139572
3,Germany,1981,29.480000,14.000000,43.480000,389.489336,11.163335,51.086029,1,ALB,...,1.808177e+09,Germany,2726056.0,2.404632e-08,ALB,5.745635,0,..,ALB,-2.139572
4,France,1981,8.580000,18.090909,26.670909,389.489336,6.847661,57.933690,1,ALB,...,1.808177e+09,France,2726056.0,1.475016e-08,ALB,5.745635,0,..,ALB,-2.139572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93035,Fiji,2023,1.461529,0.107416,1.568945,80.116893,1.958320,87.263122,1,TUV,...,6.228031e+07,Fiji,9816.0,2.519167e-08,TUV,3.852809,166290,..,TUV,6.004099
93036,United States of America,2023,0.635322,0.880947,1.516269,80.116893,1.892571,89.155693,1,TUV,...,6.228031e+07,United States of America,9816.0,2.434588e-08,TUV,3.852809,166290,..,TUV,6.004099
93037,Japan,2023,1.442782,0.000001,1.442783,80.116893,1.800847,90.956540,1,TUV,...,6.228031e+07,Japan,9816.0,2.316596e-08,TUV,3.852809,166290,..,TUV,6.004099
93038,China,2023,1.296829,0.000287,1.297116,80.116893,1.619029,92.575570,1,TUV,...,6.228031e+07,China,9816.0,2.082706e-08,TUV,3.852809,166290,..,TUV,6.004099


In [8]:

# 📁 Rutas
base_dir = r"C:\Users\51943\Videos"
archivo_completo = os.path.join(base_dir, "final", "completo_inflation.xlsx")
archivo_pob = os.path.join(base_dir, "pbi_percap_long.xlsx")
output_dir = os.path.join(base_dir, "final")
os.makedirs(output_dir, exist_ok=True)

# 📦 Cargar bases
df_completo = pd.read_excel(archivo_completo)
df_pob = pd.read_excel(archivo_pob)

# 🧼 Renombrar
df_pob.rename(columns={"Country Name": "country"}, inplace=True)

# 🧽 Reemplazos de nombres alternativos
reemplazos_nombres = {
    "Turkiye": "Turkey",
    "Iran, Islamic Rep.": "Iran",
    "Venezuela, RB": "Venezuela",
    "Vietnam": "Viet Nam",
    "North Macedonia": "Macedonia (the former Yugoslav Rep. of)",
    "Slovak Republic": "Slovakia",
    "Czechia": "Czech Republic",
    "Yemen, Rep.": "Yemen",
    "Syrian Arab Republic": "Syrian Arab Republic",
    "St. Lucia": "Saint Lucia",
    "St. Kitts and Nevis": "Saint Kitts and Nevis",
    "St. Vincent and the Grenadines": "Saint Vincent and the Grenadines",
    "Lao PDR": "Lao People's Democratic Republic",
    "Kyrgyz Republic": "Kyrgyzstan",
    "Micronesia, Fed. Sts.": "Micronesia (Federated States of)",
    "Congo, Dem. Rep.": "Congo (Democratic Republic of the)",
    "Congo, Rep.": "Congo",
    "Gambia, The": "Gambia",
    "West Bank and Gaza": "Palestine, State of",
    "Macao SAR, China": "Macao",
    "Hong Kong SAR, China": "Hong Kong",
    "Bonaire, Sint Eustatius and Saba": "Bonaire",
    "Curacao": "Curaçao",
    "Sint Maarten (Dutch part)": "Sint Maarten",
    "St. Martin (French part)": "Saint Martin (French part)",
    "Eswatini": "Swaziland",
    "Kosovo": "Kosovo",
    "Faroe Islands": "Faroe Islands",
    "American Samoa": "American Samoa",
    "Greenland": "Greenland",
    "Puerto Rico": "Puerto Rico",
    "Guam": "Guam",
    "Cabo Verde": "Cape Verde",
    "United States": "United States of America",
    "Korea, Rep.": "Korea",
    "Korea, Dem. People's Rep.": "Korea, Dem. People's Rep. of",
    "Kazakhstan": "Kazakstan",
    "Belgium": "Belgium and Luxembourg",
    "Egypt, Arab Rep.": "Egypt"
}
df_pob["country"] = df_pob["country"].replace(reemplazos_nombres)

# 🧬 1. Merge para obtener Población_j
df_merge = df_completo.merge(
    df_pob[["country", "year", "pbi_percap"]],
    how="left",
    left_on=["Country", "year"],
    right_on=["country", "year"]
).rename(columns={"pbi_percap": "pbi_percap_j"})

# 🧬 2. Merge adicional para obtener Población_i
df_merge = df_merge.merge(
    df_pob[["country_code", "year", "pbi_percap"]],
    how="left",
    left_on=["ISOi", "year"],
    right_on=["country_code", "year"]
).rename(columns={"pbi_percap": "pbi_percap_i"})

# 🧽 Limpiar columnas auxiliares


archivo_salida = os.path.join(output_dir, "completo_percap.xlsx")
df_merge.to_excel(archivo_salida, index=False)
print(f"✅ Archivo con percap_j y guardado en: {archivo_salida}")

ValueError: This sheet is too large! Your sheet size is: 10782480, 31 Max sheet size is: 1048576, 16384

In [5]:
import os
import pandas as pd

# 📁 Rutas
base_dir = r"C:\Users\51943\Videos"
archivo_completo = os.path.join(base_dir, "final", "completo_inflation.xlsx")
archivo_pob = os.path.join(base_dir, "pbi_percap_long.xlsx")
output_dir = os.path.join(base_dir, "final")
os.makedirs(output_dir, exist_ok=True)

# 📦 Cargar bases
df_completo = pd.read_excel(archivo_completo)
df_pob = pd.read_excel(archivo_pob)

# 🧼 Renombrar
df_pob.rename(columns={"Country Name": "country"}, inplace=True)

# 🧽 Reemplazos de nombres alternativos
reemplazos_nombres = {
    "Turkiye": "Turkey",
    "Iran, Islamic Rep.": "Iran",
    "Venezuela, RB": "Venezuela",
    "Vietnam": "Viet Nam",
    "North Macedonia": "Macedonia (the former Yugoslav Rep. of)",
    "Slovak Republic": "Slovakia",
    "Czechia": "Czech Republic",
    "Yemen, Rep.": "Yemen",
    "Syrian Arab Republic": "Syrian Arab Republic",
    "St. Lucia": "Saint Lucia",
    "St. Kitts and Nevis": "Saint Kitts and Nevis",
    "St. Vincent and the Grenadines": "Saint Vincent and the Grenadines",
    "Lao PDR": "Lao People's Democratic Republic",
    "Kyrgyz Republic": "Kyrgyzstan",
    "Micronesia, Fed. Sts.": "Micronesia (Federated States of)",
    "Congo, Dem. Rep.": "Congo (Democratic Republic of the)",
    "Congo, Rep.": "Congo",
    "Gambia, The": "Gambia",
    "West Bank and Gaza": "Palestine, State of",
    "Macao SAR, China": "Macao",
    "Hong Kong SAR, China": "Hong Kong",
    "Bonaire, Sint Eustatius and Saba": "Bonaire",
    "Curacao": "Curaçao",
    "Sint Maarten (Dutch part)": "Sint Maarten",
    "St. Martin (French part)": "Saint Martin (French part)",
    "Eswatini": "Swaziland",
    "Kosovo": "Kosovo",
    "Faroe Islands": "Faroe Islands",
    "American Samoa": "American Samoa",
    "Greenland": "Greenland",
    "Puerto Rico": "Puerto Rico",
    "Guam": "Guam",
    "Cabo Verde": "Cape Verde",
    "United States": "United States of America",
    "Korea, Rep.": "Korea",
    "Korea, Dem. People's Rep.": "Korea, Dem. People's Rep. of",
    "Kazakhstan": "Kazakstan",
    "Belgium": "Belgium and Luxembourg",
    "Egypt, Arab Rep.": "Egypt"
}
df_pob["country"] = df_pob["country"].replace(reemplazos_nombres)

# 🧼 Eliminar duplicados para evitar merge explosivo
df_pob_j = df_pob.drop_duplicates(subset=["country", "year"])
df_pob_i = df_pob.drop_duplicates(subset=["country_code", "year"])

# 🧬 1. Merge para obtener pbi_percap_j
df_merge = df_completo.merge(
    df_pob_j[["country", "year", "pbi_percap"]],
    how="left",
    left_on=["Country", "year"],
    right_on=["country", "year"]
).rename(columns={"pbi_percap": "pbi_percap_j"})

# 🧬 2. Merge adicional para obtener pbi_percap_i
df_merge = df_merge.merge(
    df_pob_i[["country_code", "year", "pbi_percap"]],
    how="left",
    left_on=["ISOi", "year"],
    right_on=["country_code", "year"]
).rename(columns={"pbi_percap": "pbi_percap_i"})

# 🧽 Eliminar columnas auxiliares si ya no son necesarias
df_merge.drop(columns=["country", "country_code"], inplace=True, errors='ignore')

# 💾 Guardar archivo de salida
archivo_salida = os.path.join(output_dir, "completo_percap.xlsx")
df_merge.to_excel(archivo_salida, index=False)
print(f"✅ Archivo generado: {archivo_salida}")


✅ Archivo generado: C:\Users\51943\Videos\final\completo_percap.xlsx


In [4]:

# 📁 Ruta base y archivo
base_dir = r"C:\Users\51943\Videos"
archivo_pob = os.path.join(base_dir, "poblacion.xlsx")

# 📦 Cargar datos
df = pd.read_excel(archivo_pob)

# 🔄 Convertir de wide a long
df_long = df.melt(
    id_vars=["Country Name", "Country Code"],
    var_name="year",
    value_name="poblacion"
)

# 🧼 Convertir año a entero
df_long["year"] = df_long["year"].astype(int)
df_long.rename(columns={"Country Code": "country_code"}, inplace=True)

# 💾 Guardar resultado
df_long.to_excel(os.path.join(base_dir, "poblacion_long.xlsx"), index=False)
print("✅ Población convertida de wide a long.")

✅ Población convertida de wide a long.


In [6]:
import os
import pandas as pd

# 📁 Rutas
base_dir = r"C:\Users\51943\Videos"
archivo_completo = os.path.join(base_dir, "final", "completo_percap.xlsx")
archivo_pob = os.path.join(base_dir, "poblacion_long.xlsx")
output_dir = os.path.join(base_dir, "final")
os.makedirs(output_dir, exist_ok=True)

# 📦 Cargar bases
df_completo = pd.read_excel(archivo_completo)
df_pob = pd.read_excel(archivo_pob)

# 🧼 Renombrar
df_pob.rename(columns={"Country Name": "country"}, inplace=True)

# 🧽 Reemplazos de nombres alternativos
reemplazos_nombres = {
    "Turkiye": "Turkey",
    "Iran, Islamic Rep.": "Iran",
    "Venezuela, RB": "Venezuela",
    "Vietnam": "Viet Nam",
    "North Macedonia": "Macedonia (the former Yugoslav Rep. of)",
    "Slovak Republic": "Slovakia",
    "Czechia": "Czech Republic",
    "Yemen, Rep.": "Yemen",
    "Syrian Arab Republic": "Syrian Arab Republic",
    "St. Lucia": "Saint Lucia",
    "St. Kitts and Nevis": "Saint Kitts and Nevis",
    "St. Vincent and the Grenadines": "Saint Vincent and the Grenadines",
    "Lao PDR": "Lao People's Democratic Republic",
    "Kyrgyz Republic": "Kyrgyzstan",
    "Micronesia, Fed. Sts.": "Micronesia (Federated States of)",
    "Congo, Dem. Rep.": "Congo (Democratic Republic of the)",
    "Congo, Rep.": "Congo",
    "Gambia, The": "Gambia",
    "West Bank and Gaza": "Palestine, State of",
    "Macao SAR, China": "Macao",
    "Hong Kong SAR, China": "Hong Kong",
    "Bonaire, Sint Eustatius and Saba": "Bonaire",
    "Curacao": "Curaçao",
    "Sint Maarten (Dutch part)": "Sint Maarten",
    "St. Martin (French part)": "Saint Martin (French part)",
    "Eswatini": "Swaziland",
    "Kosovo": "Kosovo",
    "Faroe Islands": "Faroe Islands",
    "American Samoa": "American Samoa",
    "Greenland": "Greenland",
    "Puerto Rico": "Puerto Rico",
    "Guam": "Guam",
    "Cabo Verde": "Cape Verde",
    "United States": "United States of America",
    "Korea, Rep.": "Korea",
    "Korea, Dem. People's Rep.": "Korea, Dem. People's Rep. of",
    "Kazakhstan": "Kazakstan",
    "Belgium": "Belgium and Luxembourg",
    "Egypt, Arab Rep.": "Egypt"
}
df_pob["country"] = df_pob["country"].replace(reemplazos_nombres)

# 🧼 Eliminar duplicados para evitar merge explosivo
df_pob_j = df_pob.drop_duplicates(subset=["country", "year"])
df_pob_i = df_pob.drop_duplicates(subset=["country_code", "year"])

# 🧬 1. Merge para obtener pbi_percap_j
df_merge = df_completo.merge(
    df_pob_j[["country", "year", "poblacion"]],
    how="left",
    left_on=["Country", "year"],
    right_on=["country", "year"]
).rename(columns={"poblacion": "poblacion_j"})

# 🧬 2. Merge adicional para obtener pbi_percap_i
df_merge = df_merge.merge(
    df_pob_i[["country_code", "year", "poblacion"]],
    how="left",
    left_on=["ISOi", "year"],
    right_on=["country_code", "year"]
).rename(columns={"poblacion": "poblacion_i"})

# 🧽 Eliminar columnas auxiliares si ya no son necesarias
df_merge.drop(columns=["country", "country_code"], inplace=True, errors='ignore')

# 💾 Guardar archivo de salida
archivo_salida = os.path.join(output_dir, "finalf.xlsx")
df_merge.to_excel(archivo_salida, index=False)
print(f"✅ Archivo generado: {archivo_salida}")

✅ Archivo generado: C:\Users\51943\Videos\final\finalf.xlsx
